In [2]:
#!/usr/bin/env python3
"""Export notebook content from Assignment2.ipynb into assignment2.py.

- Markdown cells are converted to Python comments.
- Question headings (e.g., "## Q1.2") are emitted as visible section separators.
- Code cells are copied as-is.
"""

from __future__ import annotations

import json
import re
from pathlib import Path

NOTEBOOK_PATH = Path("Assignment2.ipynb")
PYTHON_PATH = Path("assignment2.py")
QUESTION_RE = re.compile(r"^#+\s*(Q\d+(?:\.\d+)*)\b", re.IGNORECASE)


def markdown_to_comments(text: str) -> str:
    stripped = text.strip("\n")
    if not stripped:
        return ""

    lines = stripped.splitlines()
    first_non_empty = next((ln.strip() for ln in lines if ln.strip()), "")
    question_match = QUESTION_RE.match(first_non_empty)

    out: list[str] = []
    if question_match:
        question = question_match.group(1).upper()
        out.extend(
            [
                "# " + "=" * 70,
                f"# {question}",
                "# " + "=" * 70,
            ]
        )

    for line in lines:
        clean = line.rstrip()
        out.append(f"# {clean}" if clean else "#")

    return "\n".join(out)


def main() -> None:
    notebook = json.loads(NOTEBOOK_PATH.read_text())

    chunks: list[str] = []
    for cell in notebook.get("cells", []):
        source = "".join(cell.get("source", []))

        if cell.get("cell_type") == "markdown":
            comment_block = markdown_to_comments(source)
            if comment_block:
                chunks.append(comment_block)
            continue

        if cell.get("cell_type") == "code":
            stripped = source.rstrip()
            if stripped:
                chunks.append(stripped)

    output = "\n\n".join(chunks)
    if output:
        output += "\n"

    PYTHON_PATH.write_text(output)
    print(f"Updated {PYTHON_PATH} from {NOTEBOOK_PATH}.")


if __name__ == "__main__":
    main()


Updated assignment2.py from Assignment2.ipynb.
